In [ ]:
from scipy.io import loadmat
import scipy
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
import os
import pandas as pd

def normalise(X):
    for i in range(len(X[0])):
        v = X[:,i]
        X[:,i] = (v - v.min())/(v.max() - v.min())
    return X


labels = []
i=0
ccc=0
list_file_name=[]
user_data = []
user_level_understanding = []
for subdir, dirs, files in os.walk('Path_name'):
    print("user",i)
    j=0
    if i==0:
        i+=1
        continue
    
    levels_und = []
    for file in files:
        print("video ",j)
        filepath = os.path.join(subdir, file)
        X = np.genfromtxt(filepath, delimiter=',',skip_header=False)
        #X = X[:,:-2]
        print("X", X.shape)
        X = normalise(X)
        fs = 512  # Sampling rate (512 Hz)

        # Get real amplitudes of FFT (only in postive frequencies)
        fft_vals = np.absolute(np.fft.rfft(X))    #these are my fft values rfft returns only the part of the result that 
                                                  #corresponds to nonpositive frequences. (Avoids complex conjugaes) 
                                                  #faster and for plotting

        # Get frequencies for amplitudes in Hz
    
        fft_freq = np.fft.rfftfreq(len(X), 1.0 / fs)     # that might be fixed (window length n , and  sample spacing)
                                                        #inverse of the sampling rate   returns sample freq of length n .

        # Define EEG bands
        eeg_bands = {'Delta': (0, 4),
                    'Theta': (4, 8),
                    'Alpha': (8, 12),
                    'Beta': (12, 30),
                    'Gamma': (30, 45)}

        # Take the mean of the fft amplitude for each EEG band
        eeg_band_fft5 = dict()
        for band in eeg_bands:
            freq_ix = np.where((fft_freq >= eeg_bands[band][0]) &   #np.where is like asking "tell me where in this array, entries satisfy a given condition".
                        (fft_freq <= eeg_bands[band][1]))[0]    #for fft_frreq at all point where it satisfies it returns the index (in array)
                                                             #if fftfreq[np.where bla bla] will give values array
            eeg_band_fft5[band] = np.mean(fft_vals[freq_ix])
    
       # Plot the data (using pandas here cause it's easy)
        print(".................ans.........................................")
        print(eeg_band_fft5['Beta'])
        print(eeg_band_fft5['Alpha'])
        print(eeg_band_fft5['Theta'])
        print("....................ans......................................")
        print("File_name",eeg_band_fft5['Beta']/(eeg_band_fft5['Alpha'] +eeg_band_fft5['Theta']))
        print("Duration of the video:",len(X))
        print("..........................................................")
        #if ((eeg_band_fft5['Beta']/eeg_band_fft5['Alpha'])>1):
        if ((eeg_band_fft5['Beta'])/(eeg_band_fft5['Alpha'])>1):
            list_file_name.append(file)
            levels_und.append(1)
            print("1",file)
        else:
            levels_und.append(0)
            print("0",file)
        fig=plt.figure()
        fig.show()
        df = pd.DataFrame(columns=['band', 'val'])
        df['band'] = eeg_bands.keys()
        df['val'] = [eeg_band_fft5[band] for band in eeg_bands]
        ax = df.plot.bar(x='band', y='val', legend=False,color='k')
        ax.set_xlabel("EEG band of Full video")
        ax.set_ylabel("Mean band Amplitude")
        #plt.savefig(file+'.png', bbox_inches='tight')
        #plt.savefig(file+'.pdf', bbox_inches='tight')
        print("=============================================================================================")
    labels.append(levels_und)

In [ ]:
import numpy as np
np.savetxt('All_without_cluster_1sec.csv', levels_und, delimiter=",")